In [5]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import csv
import datetime as dt

In [6]:
# school
# hostip = "10.192.40.238"
# home
# hostip = "192.168.0.88"
hostip = "192.168.0.160"

topic = 'ProducerB'
file_path = 'FIT3182-A2/data/camera_event_B.csv' 


def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(data, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        print('Message published successfully. ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
        

In [7]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [8]:
if __name__ == '__main__':
    print('Publishing records..')
    producer = connect_kafka_producer()

    if producer is None:
        print('Failed to connect to Kafka. Exiting..')
        exit(1)

    current_batch = None
    batch_events = []

    with open(file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            json_row = dumps(row)
            if current_batch is None:
                current_batch = row['batch_id']

            if current_batch != row['batch_id']:
                for event in batch_events:
                    message = {
                        'event_id': event['event_id'],
                        "car_plate": event['car_plate'],
                        "camera_id": int(event['camera_id']),
                        "timestamp": event['timestamp'],
                        "speed_reading": float(event['speed_reading']),
                        "producer": "B"
                    }
                    publish_message(producer, topic, event['event_id'], dumps(message))
                producer.flush()
                print(f'Batch {current_batch} published successfully.')
                sleep(5)

                current_batch = row['batch_id']
                batch_events = []

            batch_events.append(row)

        if batch_events:
            for event in batch_events:
                message = {
                    'event_id': event['event_id'],
                    "car_plate": event['car_plate'],
                    "camera_id": int(event['camera_id']),
                    "timestamp": event['timestamp'],
                    "speed_reading": float(event['speed_reading']),
                    "producer": "B"
                }
                publish_message(producer, topic, event['event_id'], dumps(message))
            producer.flush()
            print(f'Final batch {current_batch} published successfully.')

    print('All messages published successfully.')

Publishing records..
Message published successfully. {"event_id": "b5b7dae7-3bf9-4f75-aea8-398d3bde3a41", "car_plate": "CJW 924", "camera_id": 2, "timestamp": "2024-01-01T08:00:26.029536", "speed_reading": 156.8, "producer": "B"}
Message published successfully. {"event_id": "1649c03b-cd6f-47bd-b4f2-c44fcf60524d", "car_plate": "CZ 592", "camera_id": 2, "timestamp": "2024-01-01T08:00:26.729115", "speed_reading": 163.9, "producer": "B"}
Message published successfully. {"event_id": "a9c6a4b7-a6b6-4bd7-9719-88bd895814f4", "car_plate": "UTT 229", "camera_id": 2, "timestamp": "2024-01-01T08:00:26.887170", "speed_reading": 134.1, "producer": "B"}
Message published successfully. {"event_id": "4e13ec51-02db-442c-b7aa-3a2050dd1ad1", "car_plate": "WA 0712", "camera_id": 2, "timestamp": "2024-01-01T08:00:27.601617", "speed_reading": 133.8, "producer": "B"}
Message published successfully. {"event_id": "74248dd9-c9f8-48d2-94fb-dc5610a0f735", "car_plate": "AH 8", "camera_id": 2, "timestamp": "2024-01-

KeyboardInterrupt: 